In [1]:
! pip install pyspark


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .config("hive.metastore.uris", "thrift://172.27.1.8:9083")\
      .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
      .enableHiveSupport() \
      .getOrCreate()

25/05/14 02:45:11 WARN Utils: Your hostname, codespaces-367222 resolves to a loopback address: 127.0.0.1; using 10.0.5.131 instead (on interface eth0)
25/05/14 02:45:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/14 02:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/14 02:45:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/14 02:45:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/14 02:45:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [12]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|retail_db|
+---------+



In [13]:
spark.sql("SHOW TABLES IN default").show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|           customers|      false|
|  default|customers_partiti...|      false|
+---------+--------------------+-----------+



In [ ]:
spark.sql("select * from retail_db.categories").show(truncate=False)

In [14]:
spark.sql("select * from default.customers").show(truncate=False)

25/05/14 03:02:07 WARN FileSystem: Failed to initialize fileystem hdfs://namenode:8020/user/hadoop/customers: java.lang.IllegalArgumentException: java.net.UnknownHostException: namenode


IllegalArgumentException: java.net.UnknownHostException: namenode

In [ ]:
spark.sql("select * from retail_db.departments").show(truncate=False)

In [ ]:
spark.sql("select * from retail_db.orders").show(truncate=False)

In [ ]:
spark.sql("select * from retail_db.order_items").show(truncate=False)

In [ ]:
spark.sql("select * from retail_db.products").show(truncate=False)

### Enunciado 1
Mostrar un Top 20 de clientes que mas productos compraron con sus respectivos montos.

In [ ]:
spark.sql("""SELECT
    customer_id, customer_fname, customer_lname, customer_email, sum(order_item_quantity) as quantity_item_total, sum(order_item_subtotal)as total
FROM
    retail_db.customers as c
INNER JOIN
    retail_db.orders as o
    ON c.customer_id = o.order_customer_id
INNER JOIN
    retail_db.order_items as oi
    ON o.order_id = oi.order_item_order_id
WHERE order_status <> 'CANCELED'
GROUP BY customer_id, customer_fname, customer_lname, customer_email
ORDER BY  total DESC
LIMIT 20""").show()

### Enunciado 2
Mostrar las categorías con el total de productos vendidos y los montos totales por categoría

In [ ]:
spark.sql("""SELECT
    ca.category_name, sum(order_item_quantity) as item_quantity, cast(sum(order_item_subtotal) AS INT )as total
FROM retail_db.order_items as oi
INNER JOIN
    retail_db.products as p
    ON oi.order_item_product_id = p.product_id
INNER JOIN
    retail_db.categories as ca
    ON p.product_category_id = ca.category_id
GROUP BY ca.category_name;
""").show()

### Enunciado 3
Mostrar la categoría más vendida por ciudad

In [ ]:
spark.sql("""
SELECT
    customer_city, category_name
FROM (SELECT
    customer_city, category_name, count(category_name) as quantity, DENSE_RANK () OVER (
                PARTITION BY customer_city
                ORDER BY count(category_name) DESC
            ) rank
    FROM
        retail_db.customers as c
    INNER JOIN
        retail_db.orders as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products as p
        ON oi.order_item_product_id = p.product_id
    INNER JOIN
        retail_db.categories as ca
        ON p.product_category_id = ca.category_id
    GROUP BY customer_city, category_name
    ) t
WHERE rank = 1;""").show()

### Enunciado 4
Mostrar los 5 productos más vendidos por cada ciudad y el monto recaudado

In [ ]:
spark.sql("""
SELECT
    customer_city, product_name, quantity, total
FROM (SELECT
    customer_city, product_name,sum(order_item_quantity) as quantity,sum(order_item_subtotal) as total, DENSE_RANK () OVER (
                PARTITION BY customer_city
                ORDER BY sum(order_item_quantity) DESC
            ) rank
    FROM
        retail_db.customers as c
    INNER JOIN
        retail_db.orders as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products as p
        ON oi.order_item_product_id = p.product_id
    GROUP BY customer_city, product_name
    ) t
WHERE rank < 6
ORDER BY quantity DESC;
""").show()